In [180]:
import numpy as np
import itertools

def compute_costs(indices, assignments, g_transitions, l_transitions, noises):
    """
    Computes the cost of a given assignment on the given segments in indices.
    Assignments has len(assignments) == len(indices).
    """

    assignments = np.array(assignments)

    costs = 0
    for i, j in enumerate(indices):
        #j is the index of the segment
        #i is the index of the assignment
        if l_transitions[j] == np.inf:
            if assignments[i] != 1:
                costs += np.inf
            else:
                costs += 1 + abs(assignments[i]) * noises[j] + (1 + assignments[i]) * g_transitions[j]
        elif g_transitions[j] == np.inf:
            if assignments[i] != -1:
                costs += np.inf
            else:
                costs += 1 + abs(assignments[i]) * noises[j] + (1 - assignments[i]) * l_transitions[j]
        else:
            costs += 1 + abs(assignments[i]) * noises[j] + (1 + assignments[i]) * g_transitions[j] + (1 - assignments[i]) * l_transitions[j]
    return costs


def print_cost_table(g_transitions, l_transitions, noises):
    # For each gamma_i, we have the assignments -1, 0, 1. Print a table
    # of the costs of each assignment, with each gamma_i as a row, and three columns.

    print(f"{'gamma_i':<10} {'-1':<10} {'0':<10} {'1':<10}")
    for i in range(len(g_transitions)):
        print(f"{i:<10} {compute_costs([i], [-1], g_transitions, l_transitions, noises):<10.2f} {compute_costs([i], [0], g_transitions, l_transitions, noises):<10.2f} {compute_costs([i], [1], g_transitions, l_transitions, noises):<10.2f}")


def find_relaxed_shifts(indices, g_transitions, l_transitions, noises, constraint_fn):
    n = len(indices)
    best_assignments = None
    best_cost = np.inf
    for assignments in itertools.product([-1, 0, 1], repeat=n):
        
        if not constraint_fn(assignments):
            continue

        cost = compute_costs(indices, assignments, g_transitions, l_transitions, noises)
        if cost < best_cost:
            best_cost = cost
            best_assignments = assignments
    return best_cost, best_assignments

# I have three segments, gamma_0, gamma_1, gamma_2.
m = 15000
g_transitions = np.array([1, 10, 2 * m]) # number of >= transitions in each segment
l_transitions = np.array([np.inf, 0, 1]) # number of < transitions in each segment
noises = np.array([1, 1, 1])

print_cost_table(g_transitions, l_transitions, noises)
print()

# Costs of naive assignments
print("Naive assignments")

# gamma_0 -> gamma_2
print("gamma_0 -> gamma_2", end=" ")
print(compute_costs([0, 2], [1, 1], g_transitions, l_transitions, noises))

# gamma_1 -> gamma_2
print("gamma_1 -> gamma_2", end=" ")
print(compute_costs([1, 2], [-1, 1], g_transitions, l_transitions, noises))

# Costs of relaxed assignments
print("Relaxed assignments")

# gamma_0 -> gamma_2
print("gamma_0 -> gamma_2", end=" ")
costs, shifts = find_relaxed_shifts([0, 2], g_transitions, l_transitions, noises, lambda x: x[0] <= x[1])
print(f"{costs} (shifts: {shifts})")

# gamma_1 -> gamma_2
print("gamma_1 -> gamma_2", end=" ")
costs, shifts = find_relaxed_shifts([1, 2], g_transitions, l_transitions, noises, lambda x: x[0] <= x[1])
print(f"{costs} (shifts: {shifts})")

gamma_i    -1         0          1         
0          inf        inf        4.00      
1          2.00       11.00      22.00     
2          4.00       30002.00   60002.00  

Naive assignments
gamma_0 -> gamma_2 60006.0
gamma_1 -> gamma_2 60004.0
Relaxed assignments
gamma_0 -> gamma_2 60006.0 (shifts: (1, 1))
gamma_1 -> gamma_2 6.0 (shifts: (-1, -1))


In [205]:
import cvxpy as cp

n = 10
gammas = cp.Variable(n)

constraints = [
    gammas >= -1,
    gammas <= 1,
    gammas[1] <= gammas[4],
    gammas[4] <= gammas[6],
    gammas[6] <= gammas[8],
    gammas[8] <= gammas[9],
    # the bad constraints
    gammas[2] <= gammas[4],
    gammas[3] <= gammas[6],
    gammas[5] <= gammas[8],
    gammas[7] <= gammas[9],
    # equality constraints
    gammas[2] >= 1,
    gammas[3] >= 1,
    gammas[5] >= 1,
    gammas[7] >= 1,
]

prob = cp.Problem(cp.Minimize(1), constraints)
prob.solve()
print(gammas.value)

[ 0.         -0.01783075  1.          1.          1.          1.
  1.          1.          1.          1.        ]
